In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
import os
import cv2
from numpy import asarray
from PIL import Image
from functools import partial
from keras.applications import imagenet_utils

Using TensorFlow backend.


In [2]:
### Organizing classes into training, validation, and test
print("Organizing datasets")
masked_imgs = os.listdir('datasets/compiled/with_mask_copy')
no_mask_imgs = os.listdir('datasets/compiled/without_mask_copy')

i = 0
for img in masked_imgs:
    if i < int(0.64*len(masked_imgs)):
        os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/train/mask/' + img)
        i += 1
    elif i < int(0.80*len(masked_imgs)):
        os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/validation/mask/' + img)
        i += 1
    else:
        os.rename('datasets/compiled/with_mask_copy/' + img, 'datasets/compiled/test/mask/' + img)
        i += 1

j = 0
for img in no_mask_imgs:
    if j < int(0.64*len(no_mask_imgs)):
        os.rename('datasets/compiled/without_mask_copy/' + img,'datasets/compiled/train/no-mask/' + img)
        j += 1
    elif j < int(0.80*len(no_mask_imgs)):
        os.rename('datasets/compiled/without_mask_copy/' + img, 'datasets/compiled/validation/no-mask/' + img)
        j += 1
    else:
        os.rename('datasets/compiled/without_mask_copy/' + img, 'datasets/compiled/test/no-mask/' + img)
        j += 1

print("Images reorganized")

Organizing datasets
Images reorganized


In [3]:
epochs = 2
batch_size = 16

In [4]:
# imagenet_mode_preprocessing_function = partial(imagenet_utils.preprocess_input, mode="torch")

In [5]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

validation_datagen = ImageDataGenerator(
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
        batch_size=batch_size,
		directory='datasets/compiled/train/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        batch_size=batch_size,
        directory='datasets/compiled/validation/',
        target_size=(224, 224), 
        classes = ['no-mask','mask'],
        class_mode='categorical')

Found 2452 images belonging to 2 classes.
Found 613 images belonging to 2 classes.


In [7]:
### Pre-trained Model (DenseNet121 trained on Imagenet)
model = tf.keras.applications.DenseNet121(include_top=False,weights='imagenet',input_shape=(224,224,3),classes=2)

In [8]:
# Transfer Learning
for i in model.layers:
  i.trainable = False

In [9]:
global_avg = tf.keras.layers.GlobalAveragePooling2D()(model.output)
flatten = tf.keras.layers.Flatten()(global_avg)
# drop_out = tf.keras.layers.Dropout(0.4)(flatten)
out = tf.keras.layers.Dense(2,activation='softmax')(flatten)
densenet = tf.keras.Model(inputs=[model.input],outputs=[out])

In [10]:
densenet.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [11]:
densenet.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),loss="binary_crossentropy",metrics=["accuracy"])

In [12]:
history = densenet.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
 71/154 [============>.................] - ETA: 1:45 - loss: 0.1161 - accuracy: 0.9520

/opt/anaconda3/lib/python3.7/site-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


154/154 [==============================] - 255s 2s/step - loss: 0.0755 - accuracy: 0.9682 - val_loss: 0.0140 - val_accuracy: 0.9935
Epoch 2/2
154/154 [==============================] - 273s 2s/step - loss: 0.0317 - accuracy: 0.9906 - val_loss: 0.0564 - val_accuracy: 0.9821


In [13]:
densenet.save('densenet121_detection_model.h5')

In [14]:
### Metrics
# Testing directories
test_mask_imgs = os.listdir('datasets/compiled/test/mask')
test_no_mask_imgs = os.listdir('datasets/compiled/test/no-mask')

# Numbers of images
total_mask = len(test_mask_imgs)
total_no_mask = len(test_no_mask_imgs)
total = total_mask + total_no_mask

# Confusion Matrix variables
mask_correct = 0
mask_incorrect = 0
no_mask_correct = 0
no_mask_incorrect = 0

# Mask class
for img in test_mask_imgs:
    
    tmp = Image.open('datasets/compiled/test/mask/'+ img)
    test_img = np.asarray(tmp)
    test = test_img.copy()
    test.resize(1,224,224,3)
    ans = densenet.predict(test)
    print(ans)
    if ans[0][1] > ans[0][0]:
        mask_correct += 1
        print("mask")
    elif ans[0][1] < ans[0][0]:
        mask_incorrect += 1
        print("no-mask")
        
    # Mask recall
    mask_recall = mask_correct/total_mask

# No-mask class
for img in test_no_mask_imgs:
    
    tmp = Image.open('datasets/compiled/test/no-mask/' + img)
    test_img = np.asarray(tmp)
    test = test_img.copy()
    test.resize(1,224,224,3)
    ans = densenet.predict(test)
    print(ans)
    
    if ans[0][0] > ans[0][1]:
        no_mask_correct += 1
        print("no-mask")
    elif ans[0][0] < ans[0][1]:
        no_mask_incorrect += 1
        print("mask")
        
    # No-mask recall
    no_mask_recall = no_mask_correct/total_no_mask

# Mask precision
mask_precision = (mask_correct)/(mask_correct + mask_incorrect)

# No-mask precision
no_mask_precision = (no_mask_correct)/(no_mask_correct + no_mask_incorrect)

# Mask F1 score
mask_f1 = 2 * ((mask_recall * mask_precision)/(mask_recall + mask_precision))

# No-mask F1 score
no_mask_f1 = 2 * ((no_mask_recall * no_mask_precision)/(no_mask_recall + no_mask_precision))

# Classification accuracy
accuracy = (mask_correct + no_mask_correct)/total

# Weighted recall
recall = ((mask_recall*total_mask)+(no_mask_recall*total_no_mask))/(total_mask+total_no_mask)

# Weight precision
precision = ((mask_precision*total_mask)+(no_mask_precision*total_no_mask))/(total_mask+total_no_mask)

# Weighted F1 score
f1 = 2 * ((recall * precision)/(recall + precision))

[[0.95951456 0.04048543]]
no-mask
[[1. 0.]]
no-mask
[[8.1421336e-10 1.0000000e+00]]
mask
[[1.0000000e+00 2.9101865e-17]]
no-mask
[[0.32982475 0.6701752 ]]
mask
[[8.5840135e-09 1.0000000e+00]]
mask
[[9.9999976e-01 2.1899930e-07]]
no-mask
[[1.8344545e-14 1.0000000e+00]]
mask
[[1.0000000e+00 3.4714822e-18]]
no-mask
[[0.0059531 0.9940469]]
mask
[[2.0282888e-13 1.0000000e+00]]
mask
[[1.000000e+00 6.086638e-36]]
no-mask
[[1.0000000e+00 4.7843327e-21]]
no-mask
[[9.0785761e-05 9.9990916e-01]]
mask
[[1.0000000e+00 1.9484361e-29]]
no-mask
[[1.06737806e-10 1.00000000e+00]]
mask
[[1.0000000e+00 2.4171698e-18]]
no-mask
[[0.9750088  0.02499127]]
no-mask
[[1.0876672e-20 1.0000000e+00]]
mask
[[1.0000000e+00 8.1531526e-21]]
no-mask
[[0.07343741 0.92656267]]
mask
[[1.2606309e-05 9.9998736e-01]]
mask
[[9.9999988e-01 1.5346554e-07]]
no-mask
[[1. 0.]]
no-mask
[[0.36725843 0.6327416 ]]
mask
[[1.0000000e+00 1.2820178e-24]]
no-mask
[[0.00357385 0.99642617]]
mask
[[6.397633e-07 9.999994e-01]]
mask
[[1.0000000e

[[0.10018365 0.89981633]]
mask
[[0.9979163 0.0020837]]
no-mask
[[1. 0.]]
no-mask
[[0.07519419 0.92480576]]
mask
[[7.6422234e-13 1.0000000e+00]]
mask
[[2.0994902e-13 1.0000000e+00]]
mask
[[1.000000e+00 1.876014e-32]]
no-mask
[[1.000000e+00 8.481323e-31]]
no-mask
[[1.0000000e+00 1.9561853e-25]]
no-mask
[[1.0000000e+00 3.8266613e-20]]
no-mask
[[1. 0.]]
no-mask
[[1.0000000e+00 3.0894868e-17]]
no-mask
[[9.9966609e-01 3.3395816e-04]]
no-mask
[[1. 0.]]
no-mask
[[2.5776913e-04 9.9974221e-01]]
mask
[[1.3472601e-07 9.9999988e-01]]
mask
[[1.000000e+00 9.235429e-19]]
no-mask
[[1. 0.]]
no-mask
[[1.3970517e-11 1.0000000e+00]]
mask
[[0.0028921 0.9971079]]
mask
[[1.0000000e+00 3.6132168e-13]]
no-mask
[[1. 0.]]
no-mask
[[1. 0.]]
no-mask
[[0.9980509  0.00194907]]
no-mask
[[1.316895e-14 1.000000e+00]]
mask
[[0. 1.]]
mask
[[1.0000000e+00 1.3219537e-36]]
no-mask
[[0.06046796 0.93953204]]
mask
[[2.6898306e-05 9.9997306e-01]]
mask
[[1.9487939e-13 1.0000000e+00]]
mask
[[0.00125635 0.9987437 ]]
mask
[[1.000000

[[1.0000000e+00 1.5122316e-36]]
[[1.0000000e+00 2.2170308e-31]]
[[1.000000e+00 8.218394e-26]]
[[1. 0.]]
[[1.000000e+00 1.165358e-34]]
[[1. 0.]]
[[1.0000000e+00 6.2061134e-15]]
[[1.0000000e+00 1.6479986e-22]]
[[1.0000000e+00 3.8028965e-28]]
[[1.0000000e+00 2.9308366e-28]]
[[0.362074   0.63792604]]
mask
[[1. 0.]]
[[1.000000e+00 7.656701e-27]]
[[1. 0.]]
[[1.00000000e+00 1.15186325e-14]]
[[1.0000000e+00 1.7927569e-13]]
[[1.000000e+00 2.173793e-12]]
[[1.000000e+00 4.121948e-20]]
[[1.000000e+00 6.218925e-18]]
[[1. 0.]]
[[0.5060194 0.4939806]]
[[1.3815038e-13 1.0000000e+00]]
mask
[[1.000000e+00 4.581734e-19]]
[[9.3557476e-04 9.9906439e-01]]
mask
[[1.0000000e+00 1.4173587e-26]]
[[1.0000000e+00 1.0715171e-17]]
[[0.02044684 0.97955316]]
mask
[[9.999994e-01 6.263071e-07]]
[[1. 0.]]
[[1.0000000e+00 1.0285912e-16]]
[[1. 0.]]
[[1. 0.]]
[[1.0000000e+00 1.0720816e-09]]
[[1.0000000e+00 2.5913222e-21]]
[[1. 0.]]
[[1.0000000e+00 3.5806104e-27]]
[[2.1801108e-10 1.0000000e+00]]
mask
[[1.0000000e+00 1.11977

In [15]:
print("Mask correct predictions: {}".format(mask_correct))
print("Mask incorrect predictions: {}".format(mask_incorrect))

print("No-mask correct predictions: {}".format(no_mask_correct))
print("No-mask incorrect predictions: {}".format(no_mask_incorrect))

print("Mask total images: {}".format(total_mask))
print("No-mask total images: {}".format(total_no_mask))

print("Total test images: {}".format(total))

print("Mask recall: {}".format(mask_recall))
print("No-mask recall: {}".format(no_mask_recall))

print("Mask precision: {}".format(mask_precision))
print("No-mask precision: {}".format(no_mask_precision))

print("Mask F1 score: {}".format(mask_f1))
print("No-mask F1 score: {}".format(no_mask_f1))

print("Classification accuracy: {}".format(accuracy))
print("Weighted recall: {}".format(recall))
print("Weighted precision: {}".format(precision))
print("Weighted F1 score: {}".format(f1))

Mask correct predictions: 175
Mask incorrect predictions: 209
No-mask correct predictions: 297
No-mask incorrect predictions: 87
Mask total images: 384
No-mask total images: 384
Total test images: 768
Mask recall: 0.4557291666666667
No-mask recall: 0.7734375
Mask precision: 0.4557291666666667
No-mask precision: 0.7734375
Mask F1 score: 0.45572916666666663
No-mask F1 score: 0.7734375
Classification accuracy: 0.6145833333333334
Weighted recall: 0.6145833333333334
Weighted precision: 0.6145833333333334
Weighted F1 score: 0.6145833333333334


In [ ]:
### Metrics
# Testing directories
test_mask_imgs = os.listdir('datasets/compiled/train/mask')
test_no_mask_imgs = os.listdir('datasets/compiled/train/no-mask')

# Numbers of images
total_mask = len(test_mask_imgs)
total_no_mask = len(test_no_mask_imgs)
total = total_mask + total_no_mask

# Confusion Matrix variables
mask_correct = 0
mask_incorrect = 0
no_mask_correct = 0
no_mask_incorrect = 0

# Mask class
for img in test_mask_imgs:
    
    tmp = Image.open('datasets/compiled/train/mask/'+ img)
    test_img = np.asarray(tmp)
    test = test_img.copy()
    test.resize(1,224,224,3)
    ans = densenet.predict(test)
    print(ans)
    if ans[0][1] > ans[0][0]:
        mask_correct += 1
        print("mask")
    elif ans[0][1] < ans[0][0]:
        mask_incorrect += 1
        print("no-mask")
        
    # Mask recall
    mask_recall = mask_correct/total_mask

# No-mask class
for img in test_no_mask_imgs:
    
    tmp = Image.open('datasets/compiled/train/no-mask/' + img)
    test_img = np.asarray(tmp)
    test = test_img.copy()
    test.resize(1,224,224,3)
    ans = densenet.predict(test)
    print(ans)
    
    if ans[0][0] > ans[0][1]:
        no_mask_correct += 1
        print("no-mask")
    elif ans[0][0] < ans[0][1]:
        no_mask_incorrect += 1
        print("mask")
        
    # No-mask recall
    no_mask_recall = no_mask_correct/total_no_mask

# Mask precision
mask_precision = (mask_correct)/(mask_correct + mask_incorrect)

# No-mask precision
no_mask_precision = (no_mask_correct)/(no_mask_correct + no_mask_incorrect)

# Mask F1 score
mask_f1 = 2 * ((mask_recall * mask_precision)/(mask_recall + mask_precision))

# No-mask F1 score
no_mask_f1 = 2 * ((no_mask_recall * no_mask_precision)/(no_mask_recall + no_mask_precision))

# Classification accuracy
accuracy = (mask_correct + no_mask_correct)/total

# Weighted recall
recall = ((mask_recall*total_mask)+(no_mask_recall*total_no_mask))/(total_mask+total_no_mask)

# Weight precision
precision = ((mask_precision*total_mask)+(no_mask_precision*total_no_mask))/(total_mask+total_no_mask)

# Weighted F1 score
f1 = 2 * ((recall * precision)/(recall + precision))

[[1.0000000e+00 5.5111573e-17]]
no-mask
[[3.5773857e-05 9.9996424e-01]]
mask
[[0.97621524 0.02378475]]
no-mask
[[1. 0.]]
no-mask
[[2.8536776e-06 9.9999714e-01]]
mask
[[9.9999964e-01 3.6782907e-07]]
no-mask
[[2.8386728e-08 1.0000000e+00]]
mask
[[1.000000e+00 7.817074e-27]]
no-mask
[[1.0000000e+00 4.4652373e-23]]
no-mask
[[0.00433687 0.9956631 ]]
mask
[[1. 0.]]
no-mask
[[1. 0.]]
no-mask
[[1.000000e+00 5.691362e-36]]
no-mask
[[1.0000000e+00 2.0236646e-23]]
no-mask
[[1.03734055e-04 9.99896288e-01]]
mask
[[9.9998116e-01 1.8888619e-05]]
no-mask
[[1. 0.]]
no-mask
[[9.9997902e-01 2.0990423e-05]]
no-mask
[[2.3482265e-08 1.0000000e+00]]
mask
[[1.000000e+00 3.774045e-16]]
no-mask
[[1. 0.]]
no-mask
[[1.0000000e+00 1.7557492e-17]]
no-mask
[[0.18460262 0.81539744]]
mask
[[1.0000000e+00 4.4490326e-16]]
no-mask
[[0.02982232 0.97017765]]
mask
[[9.9995255e-01 4.7461534e-05]]
no-mask
[[1.6644433e-08 1.0000000e+00]]
mask
[[1.0000000e+00 1.2999728e-17]]
no-mask
[[0.00941482 0.99058515]]
mask
[[1.000000e+00

In [19]:
print("Mask correct predictions: {}".format(mask_correct))
print("Mask incorrect predictions: {}".format(mask_incorrect))

print("No-mask correct predictions: {}".format(no_mask_correct))
print("No-mask incorrect predictions: {}".format(no_mask_incorrect))

print("Mask total images: {}".format(total_mask))
print("No-mask total images: {}".format(total_no_mask))

print("Total test images: {}".format(total))

print("Mask recall: {}".format(mask_recall))
print("No-mask recall: {}".format(no_mask_recall))

print("Mask precision: {}".format(mask_precision))
print("No-mask precision: {}".format(no_mask_precision))

print("Mask F1 score: {}".format(mask_f1))
print("No-mask F1 score: {}".format(no_mask_f1))

print("Classification accuracy: {}".format(accuracy))
print("Weighted recall: {}".format(recall))
print("Weighted precision: {}".format(precision))
print("Weighted F1 score: {}".format(f1))

Mask correct predictions: 117
Mask incorrect predictions: 158
No-mask correct predictions: 0
No-mask incorrect predictions: 0
Mask total images: 1226
No-mask total images: 1228
Total test images: 2454
Mask recall: 0.09543230016313213
No-mask recall: 0.7734375
Mask precision: 0.4557291666666667
No-mask precision: 0.7734375
Mask F1 score: 0.45572916666666663
No-mask F1 score: 0.7734375
Classification accuracy: 0.6145833333333334
Weighted recall: 0.6145833333333334
Weighted precision: 0.6145833333333334
Weighted F1 score: 0.6145833333333334


In [2]:
load_model = keras.models.load_model('densenet121_detection_model.h5')

In [3]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cv2.namedWindow("COVID Mask Detection Video Feed")
webcam = cv2.VideoCapture(0) 

classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    rval, im = webcam.read()
    im=cv2.flip(im,1,1)
    
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))
 
    faces = classifier.detectMultiScale(mini)

    for f in faces:
        (x, y, w, h) = [v * size for v in f] 
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,224,224,3))
        reshaped = np.vstack([reshaped])
        result=load_model.predict(reshaped)
        print(result)
        if result[0][0] > result[0][1]:
            percent = round(result[0][0]*100,2)
        else:
            percent = round(result[0][1]*100,2)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label] + " " + str(percent) + "%", (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    if im is not None:   
        cv2.imshow('COVID Mask Detection Video Feed', im)
    key = cv2.waitKey(10)
    
    # Exit
    if key == 27: #The Esc key
        break
        
# Stop video
webcam.release()

# Close all windows
cv2.destroyAllWindows()

[[0.9976052  0.00239478]]
[[9.9977058e-01 2.2937589e-04]]
[[9.996748e-01 3.252026e-04]]
[[9.9997687e-01 2.3170147e-05]]
[[9.9997163e-01 2.8350634e-05]]
[[9.9997461e-01 2.5426074e-05]]
[[9.9996078e-01 3.9161787e-05]]
[[9.9998987e-01 1.0132329e-05]]
[[9.9999034e-01 9.6056792e-06]]
[[9.9999332e-01 6.6239354e-06]]
[[9.9998534e-01 1.4648529e-05]]
[[9.9999630e-01 3.6626438e-06]]
[[0.9922003  0.00779965]]
[[0.6757896 0.3242104]]
[[9.9914658e-01 8.5340865e-04]]
[[0.99879885 0.00120113]]
[[0.9415395  0.05846044]]
[[0.99837273 0.00162729]]
[[9.9995923e-01 4.0713596e-05]]
[[9.996619e-01 3.381177e-04]]
[[0.99103665 0.00896339]]
[[9.9997854e-01 2.1411841e-05]]
[[9.9983203e-01 1.6792311e-04]]
[[0.9988423  0.00115778]]
[[9.9998307e-01 1.6925707e-05]]
[[9.9994850e-01 5.1487612e-05]]
[[0.99659604 0.00340393]]
[[9.9950647e-01 4.9354474e-04]]
[[9.993598e-01 6.402783e-04]]
[[9.9976951e-01 2.3053729e-04]]
[[0.9980805  0.00191948]]
[[9.9980670e-01 1.9333075e-04]]
[[9.9999797e-01 2.0477942e-06]]
[[9.9999893e

[[1.7596664e-08 1.0000000e+00]]
[[2.7561944e-08 1.0000000e+00]]
[[2.3174008e-08 1.0000000e+00]]
[[4.4290353e-08 1.0000000e+00]]
[[3.6515112e-07 9.9999964e-01]]
[[3.4389174e-08 1.0000000e+00]]
[[1.8950514e-07 9.9999976e-01]]
[[3.5018587e-07 9.9999964e-01]]
[[1.0129135e-07 9.9999988e-01]]
[[5.341489e-08 1.000000e+00]]
[[4.3981238e-08 1.0000000e+00]]
[[8.7085567e-07 9.9999917e-01]]
[[8.8466584e-07 9.9999917e-01]]
[[2.8984343e-06 9.9999714e-01]]
[[5.2761885e-07 9.9999952e-01]]
[[9.8114174e-07 9.9999905e-01]]
[[4.397763e-08 1.000000e+00]]
[[5.421033e-08 1.000000e+00]]
[[3.4971464e-07 9.9999964e-01]]
[[2.6255066e-07 9.9999976e-01]]
[[2.1413275e-07 9.9999976e-01]]
[[7.0548326e-06 9.9999297e-01]]
[[1.5457863e-07 9.9999988e-01]]
[[1.5657030e-06 9.9999845e-01]]
[[5.0239737e-06 9.9999499e-01]]
[[6.6916556e-07 9.9999928e-01]]
[[7.533348e-06 9.999925e-01]]
[[2.9248991e-05 9.9997079e-01]]
[[0.14265943 0.85734063]]
[[0.98727834 0.01272165]]
[[9.999002e-01 9.972583e-05]]
[[9.9996006e-01 3.9923645e-05]